# Manutenzione preventiva - DoS Attacks

In [1]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import warnings
import functions_notebook_new as fn
from datetime import timedelta



warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
### costanti
# clean captures
clean_capture = 'captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json'

# ping flooding captures 1 min
pf_capture_1 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0,5h_1.json'

levels_of_interest = ['frame', 'eth', 'tcp', 'arp', 'udp', 'modbus', 'icmp', 'ip']

### variabili

In [3]:
### estrazione dati
# packets clean capture
packets_cc = fn.load_packets_from_file(clean_capture)

# packets attack captures (es. packets ping flooding attack 1m per cattura di 0.5 = pkt_pf_1m_05h)
pkt_pf_1m_05h = fn.load_packets_from_file(pf_capture_1)

In [4]:
### features
# clean features
features_cc = fn.extract_features_from_packets(packets_cc, levels_of_interest)

# ping flooding attack features
features_pf_1m_05h = fn.extract_features_from_packets(pkt_pf_1m_05h, levels_of_interest)

In [5]:
df_cc = pd.DataFrame(features_cc)
df = pd.DataFrame(features_pf_1m_05h)

In [6]:
features_cc.clear()
features_pf_1m_05h.clear()

In [7]:
fn.calcolo_features_binarie(df_cc)
fn.calcolo_features_binarie(df)

In [8]:
df_cc['frame.frame.time_utc'] = pd.to_datetime(df_cc['frame.frame.time_utc'])
df['frame.frame.time_utc'] = pd.to_datetime(df['frame.frame.time_utc'])

In [ ]:
df_cc.sort_values(by='frame.frame.time_utc')
df.sort_values(by='frame.frame.time_utc')

In [ ]:
start_time = df['frame.frame.time_utc'].iloc[0]
end_time=start_time+timedelta(seconds=298.8)
end_time2 = end_time + timedelta(seconds=900)
end_time

In [ ]:
df_cc1 = df[(df['frame.frame.time_utc'] >= start_time) & (df['frame.frame.time_utc'] < end_time)].copy()
df_attack = df[(df['frame.frame.time_utc'] >= end_time) & (df['frame.frame.time_utc'] <= end_time2)].copy()
df_cc2 = df[(df['frame.frame.time_utc'] > end_time2)].copy()
df_c_full = pd.concat([df_cc, pd.concat([df_cc1, df_cc2])])
df_attack

In [ ]:
df_c_full['frame.frame.time_utc'] = pd.to_datetime(df_c_full['frame.frame.time_utc'], unit='s')

df_c_full['time_bin'] = pd.cut(df_c_full['frame.frame.time_utc'], pd.date_range(start=df_c_full['frame.frame.time_utc'].min(), 
                                                       end=df_c_full['frame.frame.time_utc'].max() + pd.Timedelta(seconds=5), 
                                                       freq='5S'))

df_features = df_c_full.groupby('time_bin').agg({
    #'dimensione_pacchetto': ['count', 'mean', 'sum', 'std'],  # Numero pacchetti, media, somma, deviazione std
    'ip.ip.src': 'nunique',  # Numero di IP unici per intervallo di tempo
    #'protocollo': 'nunique'  # Numero di protocolli unici per intervallo di tempo
}).reset_index()

df_features['time_bin']